# Simple pendulum using Lagrange's equation

Defines a LagrangianNPendulum class that is used to generate n-coupled pendulum plots from solving Lagrange's equations.

* Last revised 01-May-2020 by Sameed Pervaiz (pervaiz.8@osu.edu).

## Euler-Lagrange equation

For a double pendulum, the Lagrangian with generalized coordinates $\phi_1, \phi_2$ is

$\begin{align}
  \mathcal{L} = \frac12(m_1+m_2)L_1^2\dot\phi_1^2+m_2L_1L_2\dot\phi_1\dot\phi_2\cos(\phi_1-\phi_2)+\frac12m_2L_2^2\dot\phi_2^2 - (m_1+m_2)gL_1(1-\cos\phi_1)-m_2gL_2(1-\cos\phi_2)
\end{align}$

The Euler-Lagrange equations are

$\begin{align}
 \frac{d}{dt}\frac{\partial\mathcal{L}}{\partial \dot\phi_1} = \frac{\partial\mathcal L}{\partial\phi_1} \\
 \frac{d}{dt}\frac{\partial\mathcal{L}}{\partial \dot\phi_2} = \frac{\partial\mathcal L}{\partial\phi_2}
 \end{align}$

which tell us that

$\begin{align}
(m_1+m_2)L_1^2\ddot\phi_1 + m_2L_1L_2(\ddot\phi_2\cos(\phi_1-\phi_2)-\dot\phi_2\sin(\phi_1-\phi_2)(\dot\phi_1-\dot\phi_2)) = -m_2L_1L_2\dot\phi_1\dot\phi_2\sin(\phi_1-\phi_2)-(m_1+m_2)gL_1\sin\phi_1 \\
m_2L_1L_2(\ddot\phi_1\cos(\phi_1-\phi_2)-\dot\phi_1\sin(\phi_1-\phi_2)(\dot\phi_1-\dot\phi_2))+m_2L_2^2\ddot\phi_2 = m_2L_1L_2\dot\phi_1\dot\phi_2\sin(\phi_1-\phi_2)-m_2gL_2\sin\phi_2
\end{align}$

From here, we'll use Mathematica to solve for our accelerations.

In [ ]:
%matplotlib inline

In [ ]:
# For sin, cos, etc
import numpy as np
# integrators
from scipy.integrate import odeint, solve_ivp

# for plotting
import matplotlib.pyplot as plt

# for animations
from matplotlib import animation, rc
from IPython.display import HTML

In [ ]:
# The dpi (dots-per-inch) setting will affect the resolution and how large
#  the plots appear on screen and printed.  So you may want/need to adjust 
#  the figsize when creating the figure.
plt.rcParams['figure.dpi'] = 100.    # this is the default for notebook

# Change the common font size (smaller when higher dpi)
font_size = 10
plt.rcParams.update({'font.size': font_size})

## Pendulum class and utility functions

In [ ]:
class LagrangianDoublePendulum():
    """
    Pendulum class implements the parameters and Lagrange's equations for 
     a double pendulum (no driving or damping).
     
    Parameters
    ----------
    L1, L2 : float
        lengths of the pendulum arms
    g : float
        gravitational acceleration at the earth's surface
    m1, m2 : float
        masses of pendulum bobs

    Methods
    -------
    dy_dt(t, y)
        Returns the right side of the differential equation in vector y, 
        given time t and the corresponding value of y.
    """
    def __init__(self, L1=1., L2=1., m1=1., m2=1., g=1.
                ):
        self.L1 = L1
        self.L2 = L2
        self.g = g
        self.m1 = m1
        self.m2 = m2
    
    def dy_dt(self, t, y):
        """
        This function returns the right-hand side of the diffeq: 
        [dphi1/dt d^2phi1/dt^2 dphi2/dt d^2phi2/dt^2]
        
        Parameters
        ----------
        t : float
            time 
        y : float
            A 4-component vector [phi1(t) dphi1/dt(t) phi2(t) dphi2/dt(t)]
            
        Returns
        -------
        
        """
        L1 = self.L1
        L2 = self.L2
        g = self.g
        m1 = self.m1
        m2 = self.m2
        t1 = y[0]
        dt1 = y[1]
        t2 = y[2]
        dt2 = y[3]
        #I had Mathematica solve the 2 E-L equations for our second derivatives \dot\dot\phi_1 and \dot\dot\phi_2
        gc1 = -(g*(2.*m1+m2)*np.sin(t1)+g*m2*np.sin(t1-2.*t2)+2.*m2*(L2*dt2*dt2+L1*dt1*dt1*np.cos(t1-t2))*np.sin(t1-t2))/(2.*L1*(m1+m2-m2*np.cos(t1-t2)*np.cos(t1-t2)))
        gc2 = (L1*(m1+m2)*dt1*dt1+g*(m1+m2)*np.cos(t1)+L2*m2*dt2*dt2*np.cos(t1-t2))*np.sin(t1-t2)/(L2*(m1+m2-m2*np.cos(t1-t2)*np.cos(t1-t2)))
        return [y[1], gc1, y[3], gc2 ]
    
    def solve_ode(self, t_pts, phi1_0, phi1_dot_0, phi2_0, phi2_dot_0, 
                  abserr=1.0e-9, relerr=1.0e-9):
        """
        Solve the ODE given initial conditions.
        Specify smaller abserr and relerr to get more precision.
        """
        y = [phi1_0, phi1_dot_0, phi2_0, phi2_dot_0] 
        solution = solve_ivp(self.dy_dt, (t_pts[0], t_pts[-1]), 
                             y, t_eval=t_pts, 
                             atol=abserr, rtol=relerr)
        phi1, phi1_dot, phi2, phi2_dot = solution.y

        return phi1, phi1_dot, phi2, phi2_dot

In [ ]:
def plot_y_vs_x(x, y, axis_labels=None, label=None, title=None, 
                color=None, linestyle=None, semilogy=False, loglog=False,
                ax=None):
    """
    Generic plotting function: return a figure axis with a plot of y vs. x,
    with line color and style, title, axis labels, and line label
    """
    if ax is None:        # if the axis object doesn't exist, make one
        ax = plt.gca()

    if (semilogy):
        line, = ax.semilogy(x, y, label=label, 
                            color=color, linestyle=linestyle)
    elif (loglog):
        line, = ax.loglog(x, y, label=label, 
                          color=color, linestyle=linestyle)
    else:
        line, = ax.plot(x, y, label=label, 
                    color=color, linestyle=linestyle)

    if label is not None:    # if a label if passed, show the legend
        ax.legend()
    if title is not None:    # set a title if one if passed
        ax.set_title(title)
    if axis_labels is not None:  # set x-axis and y-axis labels if passed  
        ax.set_xlabel(axis_labels[0])
        ax.set_ylabel(axis_labels[1])

    return ax, line

In [ ]:
def start_stop_indices(t_pts, plot_start, plot_stop):
    start_index = (np.fabs(t_pts-plot_start)).argmin()  # index in t_pts array 
    stop_index = (np.fabs(t_pts-plot_stop)).argmin()  # index in t_pts array 
    return start_index, stop_index

## Make double pendulum plots 

In [ ]:
# Labels for individual plot axes
phi_vs_time_labels = (r'$t$', r'$\phi(t)$')
phi_dot_vs_time_labels = (r'$t$', r'$d\phi/dt(t)$')
state_space_labels = (r'$\phi$', r'$d\phi/dt$')

# Common plotting time (generate the full time then use slices)
t_start = 0.
t_end = 50
delta_t = 0.05

t_pts = np.arange(t_start, t_end+delta_t, delta_t)  

# Pendulum parameters
L1 = 1.
L2 = 1.
g = 1.
m1 = 1.
m2 = 1.

# Instantiate a pendulum 
p1 = LagrangianDoublePendulum(L1=L2, L2=L2, g=g, m1=m1, m2=m2)


In [ ]:
# both plots: same initial conditions
phi1_0 = (3./4.)*np.pi
phi1_dot_0 = 0.
phi2_0 = (3./4.)*np.pi
phi2_dot_0 = 0.
phi1, phi1_dot, phi2, phi2_dot = p1.solve_ode(t_pts, phi1_0, phi1_dot_0, phi2_0, phi2_dot_0)

# start the plot!
fig = plt.figure(figsize=(10,15))
overall_title = 'Double pendulum from Lagrangian:  ' + \
                rf'  $\phi_1(0) = {phi1_0:.2f},$' + \
                rf' $\dot\phi_1(0) = {phi1_dot_0:.2f}$' + \
                rf'  $\phi_2(0) = {phi2_0:.2f},$' + \
                rf' $\dot\phi_2(0) = {phi2_dot_0:.2f}$' + \
                '\n'     # \n means a new line (adds some space here)
fig.suptitle(overall_title, va='baseline')
    
# first plot: phi plot 
ax_a = fig.add_subplot(3,1,1)                  

start, stop = start_stop_indices(t_pts, t_start, t_end)    
plot_y_vs_x(t_pts[start : stop], phi1[start : stop], 
            axis_labels=phi_vs_time_labels, 
            color='blue',
            label=r'$\phi_1(t)$', 
            title=None, 
            ax=ax_a)
plot_y_vs_x(t_pts[start : stop], phi2[start : stop], 
            axis_labels=phi_vs_time_labels, 
            color='red',
            label=r'$\phi_2(t)$', 
            title=r'$\phi(t)$', 
            ax=ax_a) 
                              
# second plot: phi_dot plot 
ax_b = fig.add_subplot(3,1,2)                  

plot_y_vs_x(t_pts[start : stop], phi1_dot[start : stop], 
            axis_labels=phi_dot_vs_time_labels, 
            color='blue',
            label=r'$\dot\phi_1(t)$', 
            title=r'$\dot\phi(t)$', 
            ax=ax_b)    
plot_y_vs_x(t_pts[start : stop], phi2_dot[start : stop], 
            axis_labels=phi_dot_vs_time_labels, 
            color='red',
            label=r'$\dot\phi_2(t)$', 
            title=r'$\dot\phi(t)$', 
            ax=ax_b)

# third plot: state space plot
ax_c = fig.add_subplot(3,1,3)                  

plot_y_vs_x(phi2[start : stop], phi1_dot[start : stop], 
            axis_labels=state_space_labels, 
            color='blue',
            label=None, 
            title='State space', 
            ax=ax_c)    
plot_y_vs_x(phi2[start : stop], phi2_dot[start : stop], 
            axis_labels=state_space_labels, 
            color='red',
            label=None, 
            title='State space', 
            ax=ax_c)

fig.tight_layout()
fig.savefig('double_pendulum_Lagrange.png', bbox_inches='tight')  

Now let's do some chaos calculations! We want to show that, given two double pendulums outside the regime of small angle approximations, that the separation of their angles doesn't exponentially approach zero.

In [ ]:
# System 1
s1_phi1_0 = (3./4.)*np.pi
s1_phi1_dot_0 = 0.
s1_phi2_0 = (3./4.)*np.pi
s1_phi2_dot_0 = 0.
s1_phi1, s1_phi1_dot, s1_phi2, s1_phi2_dot = p1.solve_ode(t_pts, s1_phi1_0, s1_phi1_dot_0, s1_phi2_0, s1_phi2_dot_0)

# System 2, almost the same initial conditions, just 0.1 radians apart
s2_phi1_0 = (3./4.)*np.pi+0.1
s2_phi1_dot_0 = 0.
s2_phi2_0 = (3./4.)*np.pi+0.1
s2_phi2_dot_0 = 0.
s2_phi1, s2_phi1_dot, s2_phi2, s2_phi2_dot = p1.solve_ode(t_pts, s2_phi1_0, s2_phi1_dot_0, s2_phi2_0, s2_phi2_dot_0)

# start the plot!
fig = plt.figure(figsize=(10,10))
overall_title = 'Double pendulum from Lagrangian:  ' + \
                rf'  $\phi_1(0) = {phi1_0:.2f},$' + \
                rf' $\dot\phi_1(0) = {phi1_dot_0:.2f}$' + \
                rf'  $\phi_2(0) = {phi2_0:.2f},$' + \
                rf' $\dot\phi_2(0) = {phi2_dot_0:.2f}$' + \
                '\n'     # \n means a new line (adds some space here)
fig.suptitle(overall_title, va='baseline')
    
# first plot: phi plot comparing the two slightly-different initial conditions
ax_a = fig.add_subplot(2,2,1)                  

start, stop = start_stop_indices(t_pts, t_start, t_end)    
plot_y_vs_x(t_pts[start : stop], s1_phi1[start : stop], 
            axis_labels=phi_vs_time_labels, 
            color='blue',
            title=None, 
            label=r'System 1 $\phi_1(t)$', 
            ax=ax_a)
plot_y_vs_x(t_pts[start : stop], s2_phi1[start : stop], 
            axis_labels=phi_vs_time_labels, 
            color='red',
            title=None, 
            label=r'System 2 $\phi_1(t)$', 
            ax=ax_a)    
                              
# plot: phi plot 
ax_b = fig.add_subplot(2,2,2)                  

start, stop = start_stop_indices(t_pts, t_start, t_end)    
plot_y_vs_x(t_pts[start : stop], s1_phi2[start : stop], 
            axis_labels=phi_vs_time_labels, 
            color='blue',
            title=None, 
            label=r'System 1$\phi_2(t)$', 
            ax=ax_b)
plot_y_vs_x(t_pts[start : stop], s2_phi2[start : stop], 
            axis_labels=phi_vs_time_labels, 
            color='red',
            title=None, 
            label=r'System 2 $\phi_2(t)$', 
            ax=ax_b)    


fig.tight_layout()
fig.savefig('chaotic_trajectories.png', bbox_inches='tight')  

Okay, so far so good: the trajectories look like they're differing. For the ultimate test, let's plot the log of the difference:

In [ ]:
delta_phi1 = np.abs(s1_phi1 - s2_phi2)
delta_phi2 = np.abs(s1_phi2 - s2_phi2)

fig = plt.figure(figsize=(10,5))
overall_title = 'Test for Chaotic Motion:  ' + \
                '\n'     # \n means a new line (adds some space here)
fig.suptitle(overall_title, va='baseline')
    
# first plot: phi plot 
ax_a = fig.add_subplot(1,2,1)                  
ax_b.semilogy()

start, stop = start_stop_indices(t_pts, t_start, t_end)    
plot_y_vs_x(t_pts[start : stop], delta_phi1[start : stop], 
            axis_labels=phi_vs_time_labels, 
            color='blue',
            label=None, 
            title=r' $|\Delta\phi_1(t)|$', 
            ax=ax_a)    

ax_b = fig.add_subplot(1,2,2)
ax_b.semilogy()

start, stop = start_stop_indices(t_pts, t_start, t_end)    
plot_y_vs_x(t_pts[start : stop], delta_phi2[start : stop], 
            axis_labels=phi_vs_time_labels, 
            color='red',
            label=None, 
            title=r'$|\Delta\phi_2(t)|$', 
            ax=ax_b) 

No exponential decay! This, however, just shows sensitivity to initial conditions for just one set of initial conditions outside the small approximation regime.

We can think about what changes once we ignore small-angle approximations: we suddenly have nonlinear terms caused by the transcendental functions sine and cosine that *do not reduce to linear differential equations*. In addition, we have a 4-dimensional phase space (due to the two generalized coordinates $\phi_1, \phi_2$, along with their derivatives). Combined with the sensitivity to initial conditions, this should be enough to demonstrate chaotic motion.

# Widget time!

In [ ]:
# Import explicitly the widgets we might use (add more as needed!) 
import ipywidgets as widgets
from ipywidgets import HBox, VBox, Layout, Tab, Label, Checkbox
from ipywidgets import FloatSlider, IntSlider, Play, Dropdown, HTMLMath 

from IPython.display import display

In [ ]:
def update_plot(L1, L2, g, m1, m2, phi1_0, phi1_dot_0, phi2_0, phi2_dot_0):
    # Labels for individual plot axes
    phi_vs_time_labels = (r'$t$', r'$\phi(t)$')
    phi_dot_vs_time_labels = (r'$t$', r'$d\phi/dt(t)$')
    state_space_labels = (r'$\phi$', r'$d\phi/dt$')

    # Common plotting time (generate the full time then use slices)
    t_start = 0.
    t_end = 50
    delta_t = 0.05
    
    t_pts = np.arange(t_start, t_end+delta_t, delta_t)
    
    # Instantiate a pendulum 
    p1 = LagrangianDoublePendulum(L1=L2, L2=L2, g=g, m1=m1, m2=m2)
    
    phi1, phi1_dot, phi2, phi2_dot = p1.solve_ode(t_pts, phi1_0, phi1_dot_0, phi2_0, phi2_dot_0)
    
    # start the plot!
    fig = plt.figure(figsize=(10,15))
    overall_title = 'Double pendulum from Lagrangian:  ' + \
                rf'  $\phi_1(0) = {phi1_0:.2f},$' + \
                rf' $\dot\phi_1(0) = {phi1_dot_0:.2f}$' + \
                rf'  $\phi_2(0) = {phi2_0:.2f},$' + \
                rf' $\dot\phi_2(0) = {phi2_dot_0:.2f}$' + \
                '\n'     # \n means a new line (adds some space here)
    fig.suptitle(overall_title, va='baseline')
    
    # first plot: phi plot 
    ax_a = fig.add_subplot(3,1,1)                  

    start, stop = start_stop_indices(t_pts, t_start, t_end)    
    plot_y_vs_x(t_pts[start : stop], phi1[start : stop], 
            axis_labels=phi_vs_time_labels, 
            color='blue',
            label=r'$\phi_1(t)$', 
            title=None, 
            ax=ax_a)
    plot_y_vs_x(t_pts[start : stop], phi2[start : stop], 
            axis_labels=phi_vs_time_labels, 
            color='red',
            label=r'$\phi_2(t)$', 
            title=r'$\phi(t)$', 
            ax=ax_a) 
                              
    # second plot: phi_dot plot 
    ax_b = fig.add_subplot(3,1,2)                  

    plot_y_vs_x(t_pts[start : stop], phi1_dot[start : stop], 
            axis_labels=phi_dot_vs_time_labels, 
            color='blue',
            label=r'$\dot\phi_1(t)$', 
            title=r'$\dot\phi(t)$', 
            ax=ax_b)    
    plot_y_vs_x(t_pts[start : stop], phi2_dot[start : stop], 
            axis_labels=phi_dot_vs_time_labels, 
            color='red',
            label=r'$\dot\phi_2(t)$', 
            title=r'$\dot\phi(t)$', 
            ax=ax_b)

    # third plot: state space plot
    ax_c = fig.add_subplot(3,1,3)                  

    plot_y_vs_x(phi2[start : stop], phi1_dot[start : stop], 
            axis_labels=state_space_labels, 
            color='blue',
            label=None, 
            title='State space', 
            ax=ax_c)    
    plot_y_vs_x(phi2[start : stop], phi2_dot[start : stop], 
            axis_labels=state_space_labels, 
            color='red',
            label=None, 
            title='State space', 
            ax=ax_c)

# We need control sliders for each variable
slider_L1 = FloatSlider(value=1.0, min=0.1, max=10., step=0.1,
                      description=r'$L_1$ :')
slider_L2 = FloatSlider(value=1.0, min=0.1, max=10., step=0.1,
                      description=r'$L_2$ :')
slider_g = FloatSlider(value=1.0, min=0.1, max=10., step=0.1,
                      description=r'$g$ :')
slider_m1 = FloatSlider(value=1.0, min=0.1, max=10., step=0.1,
                      description=r'$m_1$ :')
slider_m2 = FloatSlider(value=1.0, min=0.1, max=10., step=0.1,
                      description=r'$m_2$ :')
slider_phi1_0 = FloatSlider(value=3./4.*np.pi, min=0.0, max=2*np.pi, step=0.1,
                      description=r'$\phi_1(0)$ :')
slider_dphi1_0 = FloatSlider(value=0.0, min=0.1, max=10., step=0.1,
                      description=r'$\dot\phi_1(0)$ :')
slider_phi2_0 = FloatSlider(value=3./4.*np.pi, min=0.0, max=2*np.pi, step=0.1,
                      description=r'$\phi_2(0)$ :')
slider_dphi2_0 = FloatSlider(value=0.0, min=0.1, max=10., step=0.1,
                      description=r'$\dot\phi_2(0)$ :')

# And now we plot the output with our dictionary of variable:slider correpondences
plot_out = widgets.interactive_output(update_plot,
                                      dict(L1=slider_L1, L2=slider_L2,
                                          g=slider_g, m1=slider_m1, m2=slider_m2,
                                          phi1_0=slider_phi1_0, phi1_dot_0=slider_dphi1_0,
                                          phi2_0=slider_phi2_0, phi2_dot_0=slider_dphi2_0,
                                          )
                                     )

In [ ]:
UI_box = VBox([slider_L1, slider_L2, slider_g, slider_m1,
               slider_m2, slider_phi1_0, slider_dphi1_0,
               slider_phi2_0, slider_dphi2_0, plot_out])
display(UI_box)

# Or, if you want something flashier...

In [ ]:
%%capture
x1 = L1*np.sin(phi1)
y1 = -L1*np.cos(phi1)
x2 = x1 + L2*np.sin(phi2)
y2 = y1 - L2*np.cos(phi2)

fig_anim1 = plt.figure(figsize=(5,5))
ax_anim1 = fig_anim1.add_subplot(111, xlim=(-3,3), ylim=(-3,3))
ax_anim1.set_aspect('equal')
ax_anim1.grid()
line_anim1, = ax_anim1.plot([0, x1[0], x2[0]], [0, y1[0], y2[0]], 'o-', lw=2)

def animate_double(i):
    xdata = [0, x1[i], x2[i]]
    ydata = [0, y1[i], y2[i]]
    
    line_anim1.set_data(xdata,ydata)
    return line_anim1,

anim1 = animation.FuncAnimation(fig_anim1, animate_double, init_func=None, frames=1000, interval=10, blit=True, repeat=False)

In [ ]:
HTML(anim1.to_jshtml())
# If your python env has ffmpeg, you can render a video of it, too (just change the codec if desired)
# On my (admitedly weak) machine, encoding it first tends to give smoother playback rather than the player
# anim.save('double.webm', 'ffmpeg', 30, 72, 'libvpx-vp9', 1000)

# Triple Pendulum Lagrangian

This is the truly awful part, though I will not show the even worse derivations of the E-L equations here!

$
\begin{align}
\mathcal{L} &= \frac{1}{2}\left[(m_1+m_2+m_3)L_1^2\dot\phi_1^2+(m_2+m_3)L_2^2\dot\phi_2^2+m_3L_3^2\dot\phi_3^2+2(m_2+m_3)(L_1L_2\dot\phi_1\dot\phi_2\cos(\phi_1-\phi_2))+2m_3(L_2L_3\phi_2\phi_3\cos(\phi_2-\phi_3)+L_1L_3\phi_1\phi_3\cos(\phi_1-\phi_3))\right]-g\left[(m_1+m_2+m_3)L_1(1-\cos\phi_1)+(m_2+m_3)L_2(1-\cos\phi_2)+m_3L_3(1+\cos\phi_3)\right]
\end{align}
$

In [ ]:
class LagrangianTriplePendulum():
    """
    Pendulum class implements the parameters and Lagrange's equations for 
     a double pendulum (no driving or damping).
     
    Parameters
    ----------
    L1, L2 : float
        lengths of the pendulum arms
    g : float
        gravitational acceleration at the earth's surface
    m1, m2 : float
        masses of pendulum bobs

    Methods
    -------
    dy_dt(t, y)
        Returns the right side of the differential equation in vector y, 
        given time t and the corresponding value of y.
    """
    def __init__(self, L1=1., L2=1., L3=1., m1=1., m2=1., m3=1., g=1.
                ):
        self.L1 = L1
        self.L2 = L2
        self.L3 = L3
        self.g = g
        self.m1 = m1
        self.m2 = m2
        self.m3 = m3
    
    def dy_dt(self, t, y):
        """
        This function returns the right-hand side of the diffeq: 
        [dphi1/dt d^2phi1/dt^2 dphi2/dt d^2phi2/dt^2 dphi3/dt d^2phi3/dt^2]
        
        Parameters
        ----------
        t : float
            time 
        y : float
            A 6-component vector [phi1(t) dphi1/dt(t) phi2(t) dphi2/dt(t) phi3(t) dphi3/dt(t)]
            
        Returns
        -------
        
        """
        L1 = self.L1
        L2 = self.L2
        L3 = self.L3
        g = self.g
        m1 = self.m1
        m2 = self.m2
        m3 = self.m3
        t1 = y[0]
        dt1 = y[1]
        t2 = y[2]
        dt2 = y[3]
        t3 = y[4]
        dt3 = y[5]
        # Shoutout to Mathematica for being the real MVP of this assignment
        gc1 = -((2.*g*(m2*(m2+m3)+m1*(2.*m2+m3))*np.sin(t1)+2.*g*m2*(m2+m3)*np.sin(t1-2.*t2)+4.*dt2*dt2*L2*m2*m2*np.sin(t1-t2)+4.*dt2*dt2*L2*m2*m3*np.sin(t1-t2)+2.*dt1*dt1*L1*m2*m2*np.sin(2.*(t1-t2))+2.*dt1*dt1*L1*m2*m3*np.sin(2.*(t1-t2))-g*m1*m3*np.sin(t1*2.*t2-2.*t3)+2.*dt3*dt3*L3*m2*m3*np.sin(t1-t3)+2.*dt3*dt3*L3*m2*m3*np.sin(t1-2.*t2+t3)-g*m1*m3*np.sin(t1-2.*t2+2.*t3))/(2.*L1*(2.*m1*m2+m2*m2+m1*m3+m2*m3-m2*(m2+m3)*np.cos(2.*(t1-t2))-m1*m3*np.cos(2.*(t2-t3)))))
        gc2 = -((-2.*dt1*dt1*L1*(2*m2*(m2+m3)+m1*(2*m2+m3))*np.sin(t1-t2)-2*dt2*dt2*L2*m2*(m2+m3)*np.sin(2*(t1-t2))-2*g*m1*m2*np.sin(2*t1-t2)-2*g*m2*m2*np.sin(2*t1-t2)-g*m1*m3*np.sin(2*t1-t2)-2*g*m2*m3*np.sin(2*t1-t2)+2*g*m1*m2*np.sin(t2)+2*g*m2*m2*np.sin(t2)+g*m1*m3*np.sin(t2)+2*g*m2*m3*np.sin(t2)+g*m1*m3*np.sin(t2-2*t3)+2*dt1*dt1*L1*m1*m3*np.sin(t1+t2-2*t3)+g*m1*m3*np.sin(2*t1+t2-2*t3)-2*dt3*dt3*L3*m2*m3*np.sin(2*t1-t2-t3)+4*dt3*dt3*L3*m1*m3*np.sin(t2-t3)+2*dt3*dt3*L3*m2*m3*np.sin(t2-t3)+2*dt2*dt2*L2*m1*m3*np.sin(2*(t2-t3)))/(2*L2*(2*m1*m2+m2*m2+m1*m3+m2*m3-m2*(m2+m3)*np.cos(2*(t1-t2))-m1*m3*np.cos(2*(t2-t3)))))
        gc3 = (m1*(2*dt2*dt2*L2*m2+2*dt2*dt2*L2*m3+2*dt1*dt1*L1*(m2+m3)*np.cos(t1-t2)+g*(m2+m3)*np.cos(2*t1-t2)+g*m2*np.cos(t2)+g*m3*np.cos(t2)+2*dt3*dt3*L3*m3*np.cos(t2-t3))*np.sin(t2-t3))/(L3*(2*m1*m2+m2*m2+m1*m3+m2*m3-m2*(m2+m3)*np.cos(2*(t1-t2))-m1*m3*np.cos(2*(t2-t3))))
        return [y[1], gc1, y[3], gc2, y[5], gc3 ]
    
    def solve_ode(self, t_pts, phi1_0, phi1_dot_0, phi2_0, phi2_dot_0, phi3_0, phi3_dot_0,
                  abserr=1.0e-9, relerr=1.0e-9):
        """
        Solve the ODE given initial conditions.
        Specify smaller abserr and relerr to get more precision.
        """
        y = [phi1_0, phi1_dot_0, phi2_0, phi2_dot_0, phi3_0, phi3_dot_0] 
        solution = solve_ivp(self.dy_dt, (t_pts[0], t_pts[-1]), 
                             y, t_eval=t_pts, 
                             atol=abserr, rtol=relerr)
        phi1, phi1_dot, phi2, phi2_dot, phi3, phi3_dot = solution.y

        return phi1, phi1_dot, phi2, phi2_dot, phi3, phi3_dot

In [ ]:
L1 = 1.
L2 = 1.
L3 = 1.
g = 1.
m1 = 1.
m2 = 1.
m3 = 1.

# Instantiate a pendulum 
p2 = LagrangianTriplePendulum(L1=L2, L2=L2, L3=L3, g=g, m1=m1, m2=m2, m3=m3)

In [ ]:
# both plots: same initial conditions
phi1_0 = (3./4.)*np.pi
phi1_dot_0 = 0.
phi2_0 = (3./4.)*np.pi
phi2_dot_0 = 0.
phi3_0 = (3./4.)*np.pi
phi3_dot_0 = 0.
phi1, phi1_dot, phi2, phi2_dot, phi3, phi3_dot = p2.solve_ode(t_pts, phi1_0, phi1_dot_0, phi2_0, phi2_dot_0, phi3_0, phi3_dot_0)


# start the plot!
fig = plt.figure(figsize=(10,15))
overall_title = 'Double pendulum from Lagrangian:  ' + \
                rf'  $\phi_1(0) = {phi1_0:.2f},$' + \
                rf' $\dot\phi_1(0) = {phi1_dot_0:.2f}$' + \
                rf'  $\phi_2(0) = {phi2_0:.2f},$' + \
                rf' $\dot\phi_2(0) = {phi2_dot_0:.2f}$' + \
                rf'  $\phi_3(0) = {phi3_0:.2f},$' + \
                rf' $\dot\phi_3(0) = {phi3_dot_0:.2f}$' + \
                '\n'     # \n means a new line (adds some space here)
fig.suptitle(overall_title, va='baseline')
    
# first plot: phi plot 
ax_a = fig.add_subplot(3,1,1)                  

start, stop = start_stop_indices(t_pts, t_start, t_end)    
plot_y_vs_x(t_pts[start : stop], phi1[start : stop], 
            axis_labels=phi_vs_time_labels, 
            color='blue',
            label=r'$\phi_1(t)$', 
            title=r'$\phi(t)$', 
            ax=ax_a)
plot_y_vs_x(t_pts[start : stop], phi2[start : stop], 
            axis_labels=phi_vs_time_labels, 
            color='red',
            label=r'$\phi_2(t)$', 
            title=r'$\phi(t)$', 
            ax=ax_a)
plot_y_vs_x(t_pts[start : stop], phi3[start : stop], 
            axis_labels=phi_vs_time_labels, 
            color='green',
            label=r'$\phi_3(t)$', 
            title=r'$\phi(t)$', 
            ax=ax_a)   
                              
# second plot: phi_dot plot 
ax_b = fig.add_subplot(3,1,2)                  

start, stop = start_stop_indices(t_pts, t_start, t_end)    
plot_y_vs_x(t_pts[start : stop], phi1_dot[start : stop], 
            axis_labels=phi_dot_vs_time_labels, 
            color='blue',
            label=r'$\dot\phi_1(t)$', 
            title=r'$\dot\phi(t)$', 
            ax=ax_b)    
plot_y_vs_x(t_pts[start : stop], phi2_dot[start : stop], 
            axis_labels=phi_dot_vs_time_labels, 
            color='red',
            label=r'$\dot\phi_2(t)$', 
            title=r'$\dot\phi(t)$', 
            ax=ax_b)    
plot_y_vs_x(t_pts[start : stop], phi3_dot[start : stop], 
            axis_labels=phi_dot_vs_time_labels, 
            color='green',
            label=r'$\dot\phi_3(t)$', 
            title=r'$\dot\phi(t)$', 
            ax=ax_b)    
# third plot: state space plot from t=30 to t=50   
ax_c = fig.add_subplot(3,1,3)                  

start, stop = start_stop_indices(t_pts, t_start, t_end)    
plot_y_vs_x(phi2[start : stop], phi1_dot[start : stop], 
            axis_labels=state_space_labels, 
            color='blue',
            label=None, 
            title='State space', 
            ax=ax_c)    
plot_y_vs_x(phi2[start : stop], phi2_dot[start : stop], 
            axis_labels=state_space_labels, 
            color='red',
            label=None, 
            title='State space', 
            ax=ax_c)  
plot_y_vs_x(phi2[start : stop], phi3_dot[start : stop], 
            axis_labels=state_space_labels, 
            color='green',
            label=None, 
            title='State space', 
            ax=ax_c)  

fig.tight_layout()
fig.savefig('triple_pendulum_Lagrange.png', bbox_inches='tight')  

# Triple Pendulum Widget Time!

In [ ]:
def update_triple_plot(L1, L2, L3, g, m1, m2, phi1_0, phi1_dot_0, phi2_0, phi2_dot_0, phi3_0, phi3_dot_0):
    # Labels for individual plot axes
    phi_vs_time_labels = (r'$t$', r'$\phi(t)$')
    phi_dot_vs_time_labels = (r'$t$', r'$d\phi/dt(t)$')
    state_space_labels = (r'$\phi$', r'$d\phi/dt$')

    # Common plotting time (generate the full time then use slices)
    t_start = 0.
    t_end = 50
    delta_t = 0.05
    
    t_pts = np.arange(t_start, t_end+delta_t, delta_t)
    
    # Instantiate a pendulum 
    p2 = LagrangianTriplePendulum(L1=L2, L2=L2, L3=L3, g=g, m1=m1, m2=m2)
    
    phi1, phi1_dot, phi2, phi2_dot, phi3, phi3_dot = p2.solve_ode(t_pts, phi1_0, phi1_dot_0, phi2_0, phi2_dot_0, phi3_0, phi3_dot_0)
    
    # start the plot!
    fig = plt.figure(figsize=(10,15))
    overall_title = 'Double pendulum from Lagrangian:  ' + \
                rf'  $\phi_1(0) = {phi1_0:.2f},$' + \
                rf' $\dot\phi_1(0) = {phi1_dot_0:.2f}$' + \
                rf'  $\phi_2(0) = {phi2_0:.2f},$' + \
                rf' $\dot\phi_2(0) = {phi2_dot_0:.2f}$' + \
                rf'  $\phi_3(0) = {phi3_0:.2f},$' + \
                rf' $\dot\phi_3(0) = {phi3_dot_0:.2f}$' + \
                '\n'     # \n means a new line (adds some space here)
    fig.suptitle(overall_title, va='baseline')
    
    # first plot: phi plot 
    ax_a = fig.add_subplot(3,1,1)                  

    start, stop = start_stop_indices(t_pts, t_start, t_end)    
    plot_y_vs_x(t_pts[start : stop], phi1[start : stop], 
            axis_labels=phi_vs_time_labels, 
            color='blue',
            label=r'$\phi_1(t)$', 
            title=r'$\phi(t)$', 
            ax=ax_a)
    plot_y_vs_x(t_pts[start : stop], phi2[start : stop], 
            axis_labels=phi_vs_time_labels, 
            color='red',
            label=r'$\phi_2(t)$', 
            title=r'$\phi(t)$', 
            ax=ax_a)
    plot_y_vs_x(t_pts[start : stop], phi3[start : stop], 
            axis_labels=phi_vs_time_labels, 
            color='green',
            label=r'$\phi_3(t)$', 
            title=r'$\phi(t)$', 
            ax=ax_a)   
                              
    # second plot: phi_dot plot 
    ax_b = fig.add_subplot(3,1,2)                  

    start, stop = start_stop_indices(t_pts, t_start, t_end)    
    plot_y_vs_x(t_pts[start : stop], phi1_dot[start : stop], 
            axis_labels=phi_vs_time_labels, 
            color='blue',
            label=r'$\dot\phi_1(t)$', 
            title=r'$\dot\phi(t)$', 
            ax=ax_b)    
    plot_y_vs_x(t_pts[start : stop], phi2_dot[start : stop], 
            axis_labels=phi_vs_time_labels, 
            color='red',
            label=r'$\dot\phi_2(t)$', 
            title=r'$\dot\phi(t)$', 
            ax=ax_b)    
    plot_y_vs_x(t_pts[start : stop], phi3_dot[start : stop], 
            axis_labels=phi_vs_time_labels, 
            color='green',
            label=r'$\dot\phi_3(t)$', 
            title=r'$\dot\phi(t)$', 
            ax=ax_b)    
    # third plot: state space plot from t=30 to t=50   
    ax_c = fig.add_subplot(3,1,3)                  

    start, stop = start_stop_indices(t_pts, t_start, t_end)    
    plot_y_vs_x(phi2[start : stop], phi1_dot[start : stop], 
            axis_labels=state_space_labels, 
            color='blue',
            label=None, 
            title='State space', 
            ax=ax_c)    
    plot_y_vs_x(phi2[start : stop], phi2_dot[start : stop], 
            axis_labels=state_space_labels, 
            color='red',
            label=None, 
            title='State space', 
            ax=ax_c)  
    plot_y_vs_x(phi2[start : stop], phi3_dot[start : stop], 
            axis_labels=state_space_labels, 
            color='green',
            label=None, 
            title='State space', 
            ax=ax_c)

slider_L1 = FloatSlider(value=1.0, min=0.1, max=10., step=0.1,
                      description=r'$L_1$ :')
slider_L2 = FloatSlider(value=1.0, min=0.1, max=10., step=0.1,
                      description=r'$L_2$ :')
slider_L3 = FloatSlider(value=1.0, min=0.1, max=10., step=0.1,
                      description=r'$L_3$ :')
slider_g = FloatSlider(value=1.0, min=0.1, max=10., step=0.1,
                      description=r'$g$ :')
slider_m1 = FloatSlider(value=1.0, min=0.1, max=10., step=0.1,
                      description=r'$m_1$ :')
slider_m2 = FloatSlider(value=1.0, min=0.1, max=10., step=0.1,
                      description=r'$m_2$ :')
slider_phi1_0 = FloatSlider(value=3./4.*np.pi, min=0.0, max=2*np.pi, step=0.1,
                      description=r'$\phi_1(0)$ :')
slider_dphi1_0 = FloatSlider(value=0.0, min=0.1, max=10., step=0.1,
                      description=r'$\dot\phi_1(0)$ :')
slider_phi2_0 = FloatSlider(value=3./4.*np.pi, min=0.0, max=2*np.pi, step=0.1,
                      description=r'$\phi_2(0)$ :')
slider_dphi2_0 = FloatSlider(value=0.0, min=0.1, max=10., step=0.1,
                      description=r'$\dot\phi_2(0)$ :')
slider_phi3_0 = FloatSlider(value=3./4.*np.pi, min=0.0, max=2*np.pi, step=0.1,
                      description=r'$\phi_3(0)$ :')
slider_dphi3_0 = FloatSlider(value=0.0, min=0.1, max=10., step=0.1,
                      description=r'$\dot\phi_3(0)$ :')

plot_out = widgets.interactive_output(update_triple_plot,
                                      dict(L1=slider_L1, L2=slider_L2, L3=slider_L3,
                                          g=slider_g, m1=slider_m1, m2=slider_m2,
                                          phi1_0=slider_phi1_0, phi1_dot_0=slider_dphi1_0,
                                          phi2_0=slider_phi2_0, phi2_dot_0=slider_dphi2_0,
                                          phi3_0=slider_phi3_0, phi3_dot_0=slider_dphi3_0,
                                          )
                                     )

In [ ]:
UI_box = VBox([slider_L1, slider_L2, slider_g, slider_m1,
               slider_m2, slider_phi1_0, slider_dphi1_0,
               slider_phi2_0, slider_dphi2_0,
               slider_phi3_0, slider_dphi3_0, plot_out])
display(UI_box)

# Animation Time!

In [ ]:
# Animate!
t_start = 0.
t_end = 50
delta_t = 0.05

t_pts = np.arange(t_start, t_end+delta_t, delta_t)

In [ ]:
%%capture
# Gotta calculate our coordinates relative to each arm's support
x1 = L1*np.sin(phi1)
y1 = -L1*np.cos(phi1)
x2 = x1 + L2*np.sin(phi2)
y2 = y1 - L2*np.cos(phi2)
x3 = x2 + L3*np.sin(phi3)
y3 = y2 - L3*np.cos(phi3)

fig_anim2 = plt.figure(figsize=(5,5))
ax_anim2 = fig_anim2.add_subplot(111, xlim=(-(L1+L2+L3),(L1+L2+L3)), ylim=(-(L1+L2+L3),(L1+L2+L3)))
ax_anim2.set_aspect('equal')
ax_anim2.grid()
line_anim2, = ax_anim2.plot([0, x1[0], x2[0], x3[0]], [0, y1[0], y2[0], y3[0]], 'o-', lw=2)

def animate_triple(i):
    xdata = [0, x1[i], x2[i], x3[i]]
    ydata = [0, y1[i], y2[i], y3[i]]
    
    line_anim2.set_data(xdata,ydata)
    return line_anim2,

anim2 = animation.FuncAnimation(fig_anim2, animate_triple, init_func=None, frames=1000, interval=10, blit=True, repeat=False)

In [ ]:
HTML(anim.to_jshtml())
# If your python env has ffmpeg, you can render a video of it, too (just change the codec if desired)
# anim.save('test.webm', 'ffmpeg', 30, 72, 'libvpx-vp9', 1000)

The animation turned out quite nicely; intuitively it seems we didn't make any errors in our calculation of the Euler Lagrange equations (which is nice, because revisiting them to find an error would be painful!) Impressive how effective the Lagrangian method is, though &mdash; the calculations, while large in number, aren't actually that tricky to calculate, compared to trying to set up forces to use Newton's second law.